In [7]:
import duckdb
from sqlalchemy import create_engine
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False

# Create DuckDB connection
%sql duckdb:///:memory:

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


cekk datanya sama apa engga

In [8]:
csv_data = pd.read_csv("dataset/data.csv")
csv_data

,Date,BranchID,BranchName,ProductCategory,ProductName,Quantity,Price,TotalSales,Salesperson
0,2025-01-01,BR_02,Jakarta South,Electronics,Laptop ASUS Vivobook,3,8500000,25500000,Citra
1,2025-01-01,BR_04,Bandung Main,Electronics,Smartwatch Xiaomi,3,1200000,3600000,Dewi
2,2025-01-01,BR_06,Surabaya East,Fashion,Sepatu Sneakers,3,850000,2550000,Fitri
3,2025-01-01,BR_05,Surabaya Central,Electronics,Tablet iPad Air,2,9500000,19000000,Eko
4,2025-01-01,BR_03,Jakarta West,Beauty,Parfum EDT 100ml,3,450000,1350000,Eko
...,...,...,...,...,...,...,...,...,...
5140,2025-11-10,BR_02,Jakarta South,Electronics,Headphone Sony WH-1000,2,3200000,6400000,Eko
5141,2025-02-24,BR_03,Jakarta West,Electronics,Smartphone Samsung A54,1,4500000,4500000,Joko
5142,2025-01-01,BR_07,Semarang,Electronics,Smartwatch Xiaomi,4,1200000,4800000,Dewi
5143,2025-04-09,BR_05,Surabaya Central,Home & Living,Lampu LED Smart,5,250000,1250000,Andi


In [9]:
excel_data = pd.read_excel("dataset/data.xlsx")
excel_data

,Date,BranchID,BranchName,ProductCategory,ProductName,Quantity,Price,TotalSales,Salesperson
0,2025-01-01,BR_02,Jakarta South,Electronics,Laptop ASUS Vivobook,3,8500000,25500000,Citra
1,2025-01-01,BR_04,Bandung Main,Electronics,Smartwatch Xiaomi,3,1200000,3600000,Dewi
2,2025-01-01,BR_06,Surabaya East,Fashion,Sepatu Sneakers,3,850000,2550000,Fitri
3,2025-01-01,BR_05,Surabaya Central,Electronics,Tablet iPad Air,2,9500000,19000000,Eko
4,2025-01-01,BR_03,Jakarta West,Beauty,Parfum EDT 100ml,3,450000,1350000,Eko
...,...,...,...,...,...,...,...,...,...
5140,2025-11-10,BR_02,Jakarta South,Electronics,Headphone Sony WH-1000,2,3200000,6400000,Eko
5141,2025-02-24,BR_03,Jakarta West,Electronics,Smartphone Samsung A54,1,4500000,4500000,Joko
5142,2025-01-01,BR_07,Semarang,Electronics,Smartwatch Xiaomi,4,1200000,4800000,Dewi
5143,2025-04-09,BR_05,Surabaya Central,Home & Living,Lampu LED Smart,5,250000,1250000,Andi


## Fix: upgrade openpyxl

In [10]:
%pip install "openpyxl>=3.1.0" --quiet

Note: you may need to restart the kernel to use updated packages.


c:\Users\nivan\iCloudDrive\code\ngabuburit_data\tokomart_indonesia\.venv\Scripts\python.exe: No module named pip.__main__; 'pip' is a package and cannot be directly executed


In [11]:
excel_data = pd.read_excel("dataset/data.xlsx")
print("Excel shape:", excel_data.shape)
excel_data.head()

Excel shape: (5145, 9)


,Date,BranchID,BranchName,ProductCategory,ProductName,Quantity,Price,TotalSales,Salesperson
0,2025-01-01,BR_02,Jakarta South,Electronics,Laptop ASUS Vivobook,3,8500000,25500000,Citra
1,2025-01-01,BR_04,Bandung Main,Electronics,Smartwatch Xiaomi,3,1200000,3600000,Dewi
2,2025-01-01,BR_06,Surabaya East,Fashion,Sepatu Sneakers,3,850000,2550000,Fitri
3,2025-01-01,BR_05,Surabaya Central,Electronics,Tablet iPad Air,2,9500000,19000000,Eko
4,2025-01-01,BR_03,Jakarta West,Beauty,Parfum EDT 100ml,3,450000,1350000,Eko


---
## Challenge 1: Data Cleaning

### 1.1 Import & Initial Inspection

In [12]:
df = pd.read_csv("dataset/data.csv", parse_dates=["Date"])
print("Shape:", df.shape)
print("\nDtypes:")
print(df.dtypes)
print("\nSample:")
df.head()

Shape: (5145, 9)

Dtypes:
Date               datetime64[us]
BranchID                      str
BranchName                    str
ProductCategory               str
ProductName                   str
Quantity                    int64
Price                       int64
TotalSales                  int64
Salesperson                   str
dtype: object

Sample:


,Date,BranchID,BranchName,ProductCategory,ProductName,Quantity,Price,TotalSales,Salesperson
0,2025-01-01,BR_02,Jakarta South,Electronics,Laptop ASUS Vivobook,3,8500000,25500000,Citra
1,2025-01-01,BR_04,Bandung Main,Electronics,Smartwatch Xiaomi,3,1200000,3600000,Dewi
2,2025-01-01,BR_06,Surabaya East,Fashion,Sepatu Sneakers,3,850000,2550000,Fitri
3,2025-01-01,BR_05,Surabaya Central,Electronics,Tablet iPad Air,2,9500000,19000000,Eko
4,2025-01-01,BR_03,Jakarta West,Beauty,Parfum EDT 100ml,3,450000,1350000,Eko


In [13]:
print("=== BASIC INFO ===")
df.info()
print("\n=== DESCRIBE ===")
df.describe(include="all")

=== BASIC INFO ===
<class 'pandas.DataFrame'>
RangeIndex: 5145 entries, 0 to 5144
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             5145 non-null   datetime64[us]
 1   BranchID         5145 non-null   str           
 2   BranchName       5145 non-null   str           
 3   ProductCategory  5145 non-null   str           
 4   ProductName      5145 non-null   str           
 5   Quantity         5145 non-null   int64         
 6   Price            5145 non-null   int64         
 7   TotalSales       5145 non-null   int64         
 8   Salesperson      5145 non-null   str           
dtypes: datetime64[us](1), int64(3), str(5)
memory usage: 361.9 KB

=== DESCRIBE ===


,Date,BranchID,BranchName,ProductCategory,ProductName,Quantity,Price,TotalSales,Salesperson
count,5145,5145,5145,5145,5145,5145.000000,5.145000e+03,5.145000e+03,5145
unique,NaN,10,10,4,20,NaN,NaN,NaN,10
top,NaN,BR_07,Semarang,Fashion,Tas Ransel,NaN,NaN,NaN,Eko
freq,NaN,545,545,1327,290,NaN,NaN,NaN,919
mean,2025-08-13 17:36:16.793003,NaN,NaN,NaN,NaN,2.997279,1.695335e+06,5.107806e+06,NaN
min,2025-01-01 00:00:00,NaN,NaN,NaN,NaN,-5.000000,-3.200000e+06,0.000000e+00,NaN
25%,2025-04-22 00:00:00,NaN,NaN,NaN,NaN,2.000000,3.500000e+05,7.500000e+05,NaN
50%,2025-08-14 00:00:00,NaN,NaN,NaN,NaN,3.000000,5.500000e+05,1.750000e+06,NaN
75%,2025-12-05 00:00:00,NaN,NaN,NaN,NaN,4.000000,1.200000e+06,3.600000e+06,NaN
max,2026-12-02 00:00:00,NaN,NaN,NaN,NaN,5.000000,9.500000e+06,4.750000e+07,NaN


### 1.2 Date Filter: 2025-01-01 to 2026-03-31

In [14]:
start_date = pd.Timestamp("2025-01-01")
end_date   = pd.Timestamp("2026-03-31")

before_filter = len(df)
df_filtered = df[(df["Date"] >= start_date) & (df["Date"] <= end_date)].copy()
after_filter = len(df_filtered)

print(f"Rows before date filter : {before_filter}")
print(f"Rows after  date filter : {after_filter}")
print(f"Rows removed            : {before_filter - after_filter}")
print(f"Date range in filtered  : {df_filtered['Date'].min()} to {df_filtered['Date'].max()}")

Rows before date filter : 5145
Rows after  date filter : 5140
Rows removed            : 5
Date range in filtered  : 2025-01-01 00:00:00 to 2026-03-31 00:00:00


### 1.3 Anomaly Detection

In [15]:
anomaly_log = {}

# 1. Duplicate rows (exact match)
n_dups = df_filtered.duplicated().sum()
anomaly_log["Duplicate rows (exact match)"] = int(n_dups)
print(f"Duplicate rows to remove : {n_dups}")
df_filtered[df_filtered.duplicated(keep=False)].head(6)

Duplicate rows to remove : 24


,Date,BranchID,BranchName,ProductCategory,ProductName,Quantity,Price,TotalSales,Salesperson
12,2025-01-01,BR_07,Semarang,Electronics,Smartwatch Xiaomi,4,1200000,4800000,Dewi
376,2025-02-02,BR_10,Makassar,Electronics,Laptop ASUS Vivobook,1,8500000,8500000,Gita
388,2025-02-02,BR_10,Makassar,Electronics,Laptop ASUS Vivobook,1,8500000,8500000,Gita
638,2025-02-24,BR_03,Jakarta West,Electronics,Smartphone Samsung A54,1,4500000,4500000,Joko
1008,2025-03-29,BR_02,Jakarta South,Fashion,Tas Ransel,3,550000,1650000,Citra
1016,2025-03-30,BR_06,Surabaya East,Fashion,Sepatu Sneakers,1,850000,850000,Budi


In [16]:
# 2. Negative values (Quantity, Price, TotalSales)
neg_mask = (
    (df_filtered["Quantity"]   < 0) |
    (df_filtered["Price"]      < 0) |
    (df_filtered["TotalSales"] < 0)
)
neg_rows = df_filtered[neg_mask]
anomaly_log["Negative values (Quantity/Price/TotalSales)"] = len(neg_rows)
print(f"Rows with negative values : {len(neg_rows)}")
neg_rows

Rows with negative values : 10


,Date,BranchID,BranchName,ProductCategory,ProductName,Quantity,Price,TotalSales,Salesperson
1052,2025-04-01,BR_02,Jakarta South,Electronics,Headphone Sony WH-1000,-3,3200000,9600000,Eko
1241,2025-04-18,BR_02,Jakarta South,Beauty,Foundation Maybelline,4,-180000,720000,Indah
1648,2025-05-23,BR_01,Jakarta Central,Fashion,Jaket Denim,-5,650000,3250000,Andi
2357,2025-07-26,BR_10,Makassar,Beauty,Hair Dryer Panasonic,-1,350000,350000,Fitri
2364,2025-07-27,BR_07,Semarang,Fashion,Jaket Denim,-2,650000,1300000,Budi
3167,2025-10-05,BR_06,Surabaya East,Home & Living,Blender Philips,-5,750000,3750000,Budi
3988,2025-12-18,BR_02,Jakarta South,Electronics,Headphone Sony WH-1000,2,-3200000,6400000,Indah
4391,2026-01-24,BR_04,Bandung Main,Electronics,Headphone Sony WH-1000,2,-3200000,6400000,Dewi
4607,2026-02-11,BR_03,Jakarta West,Electronics,Smartwatch Xiaomi,-4,1200000,4800000,Eko
4839,2026-03-04,BR_03,Jakarta West,Home & Living,Kipas Angin Standing,1,-350000,350000,Budi


In [17]:
# 3. Future dates (Date > today = 2026-02-21)
today = pd.Timestamp("2026-02-21")
future_mask = df_filtered["Date"] > today
future_rows = df_filtered[future_mask]
anomaly_log["Future dates (> 2026-02-21)"] = len(future_rows)
print(f"Rows with future dates : {len(future_rows)}")
future_rows.sort_values("Date")

Rows with future dates : 411


,Date,BranchID,BranchName,ProductCategory,ProductName,Quantity,Price,TotalSales,Salesperson
4715,2026-02-22,BR_01,Jakarta Central,Electronics,Laptop ASUS Vivobook,5,8500000,42500000,Andi
4729,2026-02-22,BR_05,Surabaya Central,Electronics,Smartwatch Xiaomi,3,1200000,3600000,Eko
4728,2026-02-22,BR_07,Semarang,Electronics,Smartphone Samsung A54,5,4500000,22500000,Indah
4727,2026-02-22,BR_03,Jakarta West,Fashion,Kemeja Formal Pria,2,350000,700000,Joko
4726,2026-02-22,BR_05,Surabaya Central,Beauty,Skincare Set Korea,4,550000,2200000,Gita
...,...,...,...,...,...,...,...,...,...
5120,2026-03-31,BR_07,Semarang,Beauty,Foundation Maybelline,4,180000,720000,Budi
5121,2026-03-31,BR_02,Jakarta South,Home & Living,Lampu LED Smart,3,250000,750000,Indah
5122,2026-03-31,BR_02,Jakarta South,Beauty,Foundation Maybelline,5,180000,900000,Citra
5123,2026-03-31,BR_07,Semarang,Electronics,Smartwatch Xiaomi,1,1200000,1200000,Indah


In [18]:
# 4. Price = 0 or TotalSales = 0 while Quantity > 0
zero_mask = (
    ((df_filtered["Price"] == 0) | (df_filtered["TotalSales"] == 0))
    & (df_filtered["Quantity"] > 0)
)
zero_rows = df_filtered[zero_mask]
anomaly_log["Price/TotalSales = 0 with transaction"] = len(zero_rows)
print(f"Rows with zero price/sales : {len(zero_rows)}")
zero_rows

Rows with zero price/sales : 8


,Date,BranchID,BranchName,ProductCategory,ProductName,Quantity,Price,TotalSales,Salesperson
242,2025-01-21,BR_02,Jakarta South,Fashion,Dress Casual Wanita,4,0,0,Budi
954,2025-03-23,BR_06,Surabaya East,Fashion,Tas Ransel,4,0,0,Fitri
1533,2025-05-12,BR_05,Surabaya Central,Fashion,Dress Casual Wanita,3,0,0,Eko
2849,2025-09-07,BR_07,Semarang,Beauty,Parfum EDT 100ml,3,0,0,Budi
3149,2025-10-04,BR_07,Semarang,Home & Living,Rice Cooker Miyako,2,0,0,Dewi
4569,2026-02-08,BR_10,Makassar,Beauty,Hair Dryer Panasonic,3,0,0,Gita
4711,2026-02-21,BR_10,Makassar,Beauty,Parfum EDT 100ml,2,0,0,Hendra
4793,2026-03-01,BR_07,Semarang,Beauty,Hair Dryer Panasonic,1,0,0,Dewi


In [19]:
# 5. Date gaps (missing dates in the range)
all_dates     = pd.date_range(start_date, end_date, freq="D")
present_dates = df_filtered["Date"].dt.normalize().unique()
missing_dates = all_dates.difference(present_dates)
anomaly_log["Date gaps (missing dates)"] = len(missing_dates)
print(f"Missing dates in range : {len(missing_dates)}")
print(missing_dates.tolist())

Missing dates in range : 3
[Timestamp('2025-07-05 00:00:00'), Timestamp('2026-03-18 00:00:00'), Timestamp('2026-03-30 00:00:00')]


In [20]:
# 6. TotalSales mismatch (Price * Quantity != TotalSales)
mismatch_mask = df_filtered["Price"] * df_filtered["Quantity"] != df_filtered["TotalSales"]
mismatch_rows = df_filtered[mismatch_mask]
anomaly_log["TotalSales mismatch (Price x Qty != TotalSales)"] = len(mismatch_rows)
print(f"Rows with TotalSales mismatch : {len(mismatch_rows)}")
mismatch_rows.head()

Rows with TotalSales mismatch : 10


,Date,BranchID,BranchName,ProductCategory,ProductName,Quantity,Price,TotalSales,Salesperson
1052,2025-04-01,BR_02,Jakarta South,Electronics,Headphone Sony WH-1000,-3,3200000,9600000,Eko
1241,2025-04-18,BR_02,Jakarta South,Beauty,Foundation Maybelline,4,-180000,720000,Indah
1648,2025-05-23,BR_01,Jakarta Central,Fashion,Jaket Denim,-5,650000,3250000,Andi
2357,2025-07-26,BR_10,Makassar,Beauty,Hair Dryer Panasonic,-1,350000,350000,Fitri
2364,2025-07-27,BR_07,Semarang,Fashion,Jaket Denim,-2,650000,1300000,Budi


### 1.4 Anomaly Summary Table

In [21]:
anomaly_summary = pd.DataFrame(
    list(anomaly_log.items()),
    columns=["Anomaly Type", "Count"]
)
print("=== ANOMALY DOCUMENTATION ===")
anomaly_summary

=== ANOMALY DOCUMENTATION ===


,Anomaly Type,Count
0,Duplicate rows (exact match),24
1,Negative values (Quantity/Price/TotalSales),10
2,Future dates (> 2026-02-21),411
3,Price/TotalSales = 0 with transaction,8
4,Date gaps (missing dates),3
5,TotalSales mismatch (Price x Qty != TotalSales),10


### 1.5 Data Cleaning — Remove Anomalies

In [22]:
df_clean = df_filtered.copy()
original_len = len(df_clean)

# Step 1: Remove duplicates
df_clean = df_clean.drop_duplicates()
print(f"After removing duplicates : {len(df_clean)} rows (removed {original_len - len(df_clean)})")

# Step 2: Remove negative values
neg_mask2 = (
    (df_clean["Quantity"]   < 0) |
    (df_clean["Price"]      < 0) |
    (df_clean["TotalSales"] < 0)
)
before = len(df_clean)
df_clean = df_clean[~neg_mask2]
print(f"After removing negatives  : {len(df_clean)} rows (removed {before - len(df_clean)})")

# Step 3: Remove future dates
today = pd.Timestamp("2026-02-21")
before = len(df_clean)
df_clean = df_clean[df_clean["Date"] <= today]
print(f"After removing futures    : {len(df_clean)} rows (removed {before - len(df_clean)})")

# Step 4: Remove zero-price/sales rows
zero_mask2 = (
    ((df_clean["Price"] == 0) | (df_clean["TotalSales"] == 0))
    & (df_clean["Quantity"] > 0)
)
before = len(df_clean)
df_clean = df_clean[~zero_mask2]
print(f"After removing zero price : {len(df_clean)} rows (removed {before - len(df_clean)})")

print(f"\nFinal clean dataset: {len(df_clean)} rows")
df_clean.head()

After removing duplicates : 5116 rows (removed 24)
After removing negatives  : 5106 rows (removed 10)
After removing futures    : 4699 rows (removed 407)
After removing zero price : 4692 rows (removed 7)

Final clean dataset: 4692 rows


,Date,BranchID,BranchName,ProductCategory,ProductName,Quantity,Price,TotalSales,Salesperson
0,2025-01-01,BR_02,Jakarta South,Electronics,Laptop ASUS Vivobook,3,8500000,25500000,Citra
1,2025-01-01,BR_04,Bandung Main,Electronics,Smartwatch Xiaomi,3,1200000,3600000,Dewi
2,2025-01-01,BR_06,Surabaya East,Fashion,Sepatu Sneakers,3,850000,2550000,Fitri
3,2025-01-01,BR_05,Surabaya Central,Electronics,Tablet iPad Air,2,9500000,19000000,Eko
4,2025-01-01,BR_03,Jakarta West,Beauty,Parfum EDT 100ml,3,450000,1350000,Eko


---
## Challenge 2: Trend Analysis

### 2.1 Most Profitable Branch

In [23]:
branch_sales = (
    df_clean.groupby(["BranchID", "BranchName"])["TotalSales"]
    .sum()
    .reset_index()
    .sort_values("TotalSales", ascending=False)
    .rename(columns={"TotalSales": "TotalRevenue"})
)
branch_sales["Rank"] = range(1, len(branch_sales) + 1)
branch_sales["TotalRevenue_fmt"] = branch_sales["TotalRevenue"].apply(
    lambda x: f"Rp {x:,.0f}"
)
print("=== BRANCH PROFITABILITY ===")
print(f"Top    : {branch_sales.iloc[0]['BranchName']} — {branch_sales.iloc[0]['TotalRevenue_fmt']}")
print(f"Worst  : {branch_sales.iloc[-1]['BranchName']} — {branch_sales.iloc[-1]['TotalRevenue_fmt']}")
branch_sales[["Rank","BranchName","TotalRevenue_fmt"]]

=== BRANCH PROFITABILITY ===
Top    : Jakarta South — Rp 2,795,770,000
Worst  : Medan — Rp 2,162,290,000


,Rank,BranchName,TotalRevenue_fmt
1,1,Jakarta South,"Rp 2,795,770,000"
6,2,Semarang,"Rp 2,540,660,000"
9,3,Makassar,"Rp 2,476,020,000"
7,4,Yogyakarta,"Rp 2,459,700,000"
2,5,Jakarta West,"Rp 2,452,880,000"
5,6,Surabaya East,"Rp 2,366,820,000"
4,7,Surabaya Central,"Rp 2,361,410,000"
0,8,Jakarta Central,"Rp 2,217,950,000"
3,9,Bandung Main,"Rp 2,163,250,000"
8,10,Medan,"Rp 2,162,290,000"


### 2.2 Best-Selling Products

In [24]:
product_sales = (
    df_clean.groupby(["ProductCategory", "ProductName"])
    .agg(TotalQty=("Quantity","sum"), TotalRev=("TotalSales","sum"))
    .reset_index()
    .sort_values("TotalRev", ascending=False)
)
product_sales["TotalRev_fmt"] = product_sales["TotalRev"].apply(lambda x: f"Rp {x:,.0f}")

print("=== TOP 10 PRODUCTS BY REVENUE ===")
product_sales.head(10)[["ProductCategory","ProductName","TotalQty","TotalRev_fmt"]]

=== TOP 10 PRODUCTS BY REVENUE ===


,ProductCategory,ProductName,TotalQty,TotalRev_fmt
9,Electronics,Tablet iPad Air,658,"Rp 6,251,000,000"
6,Electronics,Laptop ASUS Vivobook,698,"Rp 5,933,000,000"
7,Electronics,Smartphone Samsung A54,731,"Rp 3,289,500,000"
5,Electronics,Headphone Sony WH-1000,756,"Rp 2,419,200,000"
8,Electronics,Smartwatch Xiaomi,679,"Rp 814,800,000"
19,Home & Living,Vacuum Cleaner,632,"Rp 758,400,000"
13,Fashion,Sepatu Sneakers,757,"Rp 643,450,000"
15,Home & Living,Blender Philips,695,"Rp 521,250,000"
11,Fashion,Jaket Denim,738,"Rp 479,700,000"
14,Fashion,Tas Ransel,776,"Rp 426,800,000"


In [25]:
cat_sales = (
    df_clean.groupby("ProductCategory")["TotalSales"]
    .sum()
    .sort_values(ascending=False)
    .reset_index()
)
cat_sales["TotalSales_fmt"] = cat_sales["TotalSales"].apply(lambda x: f"Rp {x:,.0f}")
print("=== SALES BY PRODUCT CATEGORY ===")
cat_sales[["ProductCategory","TotalSales_fmt"]]

=== SALES BY PRODUCT CATEGORY ===


,ProductCategory,TotalSales_fmt
0,Electronics,"Rp 18,707,500,000"
1,Fashion,"Rp 2,123,450,000"
2,Home & Living,"Rp 2,001,900,000"
3,Beauty,"Rp 1,163,900,000"


### 2.3 Seasonal & Time Trends

In [26]:
df_clean["YearMonth"] = df_clean["Date"].dt.to_period("M")
monthly = (
    df_clean.groupby("YearMonth")["TotalSales"]
    .sum()
    .reset_index()
)
monthly["TotalSales_fmt"] = monthly["TotalSales"].apply(lambda x: f"Rp {x:,.0f}")
print("=== MONTHLY SALES TREND ===")
monthly[["YearMonth","TotalSales_fmt"]]

=== MONTHLY SALES TREND ===


,YearMonth,TotalSales_fmt
0,2025-01,"Rp 1,743,360,000"
1,2025-02,"Rp 1,494,180,000"
2,2025-03,"Rp 1,781,310,000"
3,2025-04,"Rp 1,867,610,000"
4,2025-05,"Rp 1,783,400,000"
5,2025-06,"Rp 1,771,730,000"
6,2025-07,"Rp 1,785,390,000"
7,2025-08,"Rp 1,670,220,000"
8,2025-09,"Rp 1,620,150,000"
9,2025-10,"Rp 2,081,110,000"


In [27]:
day_map = {0:"Senin",1:"Selasa",2:"Rabu",3:"Kamis",4:"Jumat",5:"Sabtu",6:"Minggu"}
df_clean["DayOfWeek"] = df_clean["Date"].dt.dayofweek
df_clean["DayName"]   = df_clean["DayOfWeek"].map(day_map)

dow_sales = (
    df_clean.groupby(["DayOfWeek","DayName"])["TotalSales"]
    .sum()
    .reset_index()
    .sort_values("DayOfWeek")
)
dow_sales["TotalSales_fmt"] = dow_sales["TotalSales"].apply(lambda x: f"Rp {x:,.0f}")
print("=== SALES BY DAY OF WEEK ===")
print(f"Highest: {dow_sales.loc[dow_sales['TotalSales'].idxmax(), 'DayName']}")
print(f"Lowest : {dow_sales.loc[dow_sales['TotalSales'].idxmin(), 'DayName']}")
dow_sales[["DayName","TotalSales_fmt"]]

=== SALES BY DAY OF WEEK ===
Highest: Sabtu
Lowest : Kamis


,DayName,TotalSales_fmt
0,Senin,"Rp 3,050,490,000"
1,Selasa,"Rp 3,217,590,000"
2,Rabu,"Rp 3,301,630,000"
3,Kamis,"Rp 2,892,070,000"
4,Jumat,"Rp 3,299,520,000"
5,Sabtu,"Rp 4,253,210,000"
6,Minggu,"Rp 3,982,240,000"


In [28]:
sp_sales = (
    df_clean.groupby("Salesperson")["TotalSales"]
    .sum()
    .reset_index()
    .sort_values("TotalSales", ascending=False)
)
sp_sales["TotalSales_fmt"] = sp_sales["TotalSales"].apply(lambda x: f"Rp {x:,.0f}")
print("=== SALESPERSON PERFORMANCE ===")
sp_sales[["Salesperson","TotalSales_fmt"]]

=== SALESPERSON PERFORMANCE ===


,Salesperson,TotalSales_fmt
4,Eko,"Rp 4,411,570,000"
1,Budi,"Rp 3,759,740,000"
8,Indah,"Rp 3,065,260,000"
6,Gita,"Rp 2,698,730,000"
5,Fitri,"Rp 2,390,050,000"
7,Hendra,"Rp 2,112,430,000"
3,Dewi,"Rp 1,961,730,000"
0,Andi,"Rp 1,846,770,000"
9,Joko,"Rp 1,108,030,000"
2,Citra,"Rp 642,440,000"


### 2.4 Insights
- Perhatikan bulan dengan penjualan tertinggi/terendah
- Cek apakah hari kerja vs weekend mempengaruhi penjualan
- Identifikasi produk/kategori yang mendominasi revenue

---
## Challenge 3: Final Numbers

In [29]:
# Angka kunci setelah data cleaning — harus EXACT MATCH
total_quantity    = int(df_clean["Quantity"].sum())
total_sales       = int(df_clean["TotalSales"].sum())
total_salesperson = int(df_clean["Salesperson"].nunique())

print("=" * 45)
print(f"  Quantity Total     : {total_quantity:,}")
print(f"  TotalSales Total   : Rp {total_sales:,.0f}")
print(f"  Salesperson Total  : {total_salesperson}")
print("=" * 45)

  Quantity Total     : 14,095
  TotalSales Total   : Rp 23,996,750,000
  Salesperson Total  : 10
